## Aspire + AlphaCSC pipeline

In this notebook we will run automated spike detection pipeline and visualize the results. We will use simulation described in `0_simulation.ipynb`.
This notebook pursues the following goals:

1. to run the full detection pipeline using the combination of ASPIRE and aSCS algorithms;
2. to visualize each step using an interactive visualision tool `panel`.

NOTE: To import `simulation` the working directory should be changed if the example is run from the cloned GitHub repository.


In [ ]:
import os
from pathlib import Path

import matplotlib.pylab as plt
import mne
import numpy as np
mne.viz.set_3d_backend('pyvista')

# change to the root directory of the project
if os.getcwd().split("/")[-1] == "examples":
    os.chdir('..')

from megspikes.simulation.simulation import Simulation

# Setup the path for the simulation
sample_path = Path(os.getcwd()) / 'examples' / 'data' / '2_aspire_alphacsc_pipeline'
sample_path.mkdir(exist_ok=True, parents=True)

import logging
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.debug("test debug")
logging.info("test info")
%load_ext autoreload
%autoreload 2

## Simulate data

For more information see `0_simulation.ipynb`

In [ ]:
sim = Simulation(sample_path)
# sim = Simulation(sample_path, n_events=[15, 15, 0, 0])
sim.simulate_dataset()

sim

## Run Aspire-AlphaCSC Pipeline

In [ ]:
from megspikes.pipeline import aspire_alphacsc_pipeline

from sklearn import set_config
set_config(display='diagram')
set_config(print_changed_only=False)

First, we should specify pipeline parameters. All parameters are listed in the `aspire_alphacsc_default_params.yml`.

In [ ]:
params = {
    'n_ica_components': 5,
    'n_runs': 3,
    'runs': [0, 1, 2],
    'n_atoms': 2,
    'PeakDetection': {'width': 2},
    'CleanDetections': {'n_cleaned_peaks': 50},
    'SelectAlphacscEvents': {
        'z_hat_threshold': 1.,
        'z_hat_threshold_min': 0.1}
}

Using these parameters we create `sklear.Pipeline`

In [ ]:
pipe = aspire_alphacsc_pipeline(sim.case_manager, params)
pipe

We can also save it as an __HTML__ document

In [ ]:
from sklearn.utils import estimator_html_repr
with open('my_estimator.html', 'w') as f:  
    f.write(estimator_html_repr(pipe))

No input is needed to run this pipeline, so we can run it with `X=None`. As an output we acquire `xr.Dataset` with all results and `mne.io.Raw` file.

In [ ]:
dataset, raw = pipe.fit_transform(None)

In [ ]:
dataset

Now we can also load the created dataset.

In [ ]:
import xarray as xr
dataset = xr.open_dataset(sample_path / 'sample' / 'sample_alphacsc_results.nc')

# Visualize detection
To visualize the steps of ASPIRE AlphaCSC pipleine we should creat `DetectionsViewer` object.
For the interactive visualization we are using [Panel](https://panel.holoviz.org/index.html). Created instance of interactive application -- called `app` in this notebook -- could be opened in the notebook by calling the cell with app object or in the different Web Browser window by calling `app.show()`. All the created applications related to the `DetectionsViewer` are connected and updated simultaneouly with the change of the parameters. 

In [ ]:
from megspikes.visualization.visualization import DetectionsViewer
pp = DetectionsViewer(dataset, sim.case_manager)

## Plot ICA components
First part of the detection procedure is based on the ICA decomposition of the raw MEG data. So it would be helpful to visualize ICA components distribution over the sensors.

In [ ]:
app = pp.view_ica()
# app.show()
app

## Plot ICA sources and detections
To visualize ICA peak detection and selection procedure we plot detected events over the ICA sources timeseries. Note, that we can also view the peaks that were selected after AlphaCSC decompositions and peaks selection.

In [ ]:
app = pp.view_ica_sources_and_peaks()
# app.show()
app

## Plot ICA fast RAP-MUSIC localizatons
ICA peaks are selected base on results of the RAP-MUSIC algorithm. We can also plot the localization of the ICA peaks in the brain. Note, that the spacio-temporal clusters of the ICA peaks are used to generate final clusters in the original [ASPIRE](https://github.com/kuznesashka/ASPIRE.git) algorithm.

In [ ]:
app = pp.view_ica_peak_localizations()
app

## Plot AlphaCSC atoms
As a result of AlphaCSC decomposition we have spacial portrait of an atom $u$ and a temporal timeseries $v$ which we will plot in this section.

In [ ]:
pp.data.alphacsc_atoms

In [ ]:
app = pp.view_alphacsc_atoms()
# app.show()
app

## Plot AlphaCSC selected evets
Finally, we would like to veiw the generated AlphaCSC atoms and the individual events included in each atom.

In [ ]:
app = pp.view_alphacsc_clusters(sim.raw_simulation.copy())
# app.show()
app

# Localize detected clusters

In [ ]:
from megspikes.pipeline import read_detection_iz_prediction_pipeline
from sklearn import set_config
set_config(display='diagram')
set_config(print_changed_only=False)

In [ ]:
params = {
    'PrepareClustersDataset': {'detection_sfreq': 200.}
}
pipe = read_detection_iz_prediction_pipeline(sim.case_manager, params)
pipe

In [ ]:
clusters, raw = pipe.fit_transform((dataset, sim.raw_simulation.copy()))

In [ ]:
clusters

In [ ]:
import xarray as xr
clusters = xr.open_dataset(sample_path / 'sample' / 'sample_alphacsc_cluster_results.nc')

## View clusters using Cluster Slope Viewer

In [ ]:
from megspikes.visualization.visualization import ClusterSlopeViewer

In [ ]:
%matplotlib qt5
pc = ClusterSlopeViewer(clusters, sim.case_manager)
pc.data.clusters

In [ ]:
pc.view()

In [ ]:
import xarray as xr
clusters = xr.open_dataset(sample_path / 'sample' / 'sample_clusters_manually_checked.nc')

## Evaluate Irritative Zone prediction

In [ ]:
from megspikes.scoring.scoring import ScoreIZPrediction
scorer = ScoreIZPrediction()

In [ ]:
dist = scorer.score(clusters, sim.mni_resection, 'peak')
print(f"Average distance from the resection area is {dist:.0f} mm for the 'peak' condition")

In [ ]:
dist = scorer.score(clusters, sim.mni_resection, 'slope')
print(f"Average distance from the resection area is {dist:.0f} mm for the 'slope' condition")

### Plot resection and prediction

In [ ]:
%matplotlib inline
from nilearn import plotting

dist = scorer.score(clusters, sim.mni_resection, 'peak')


fig, ax = plt.subplots(figsize=(15, 7))

display = plotting.plot_glass_brain(
            None, display_mode='lzry', figure=fig, axes=ax)

display.add_markers(scorer.detection_mni, marker_color='tomato', alpha=0.2)
display.add_markers(sim.mni_resection, marker_color='indigo', alpha=0.6)